# GCC118 - Programação Matemática
## Universidade Federal de Lavras
### Problema 3
#### Aluno: Eduardo Cesar Cauduro Coelho
#### Matrícula: 202310175
#### Aluno: Felipe Geraldo de Oliveira
#### Matrícula: 202310174

## Instalação e importação da biblioteca PuLP


In [1]:
!pip install pulp
import pulp as pl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 33.6 MB/s eta 0:00:00


## Declaração dos parâmetros

* $cd \in \mathbb{Z}_+$: custo de demolição de uma casa.     
* $n \in \mathbb{Z}_+$: numero de casas.      
* $t \in \mathbb{R}_+$: tamanho do lote de uma casa.
* $D = \{s, d, t, q\}$: categorias de domicílios (simples, duplo, triplo, quádruplo).  
* $T_d \in \mathbb{R}_+$: tamanho do lote de cada tipo de domicílio $d \in D$.
* $ptq \in \mathbb{R}_+$: percentual mínimo de domicílios triplos + quadruplos.
* $ps \in \mathbb{R}_+$: percentual mínimo de domicílios simples.
* $pd \in \mathbb{R}_+$: percentual mínimo de domicílios duplos.
* $I_d \in \mathbb{R}_+$: imposto atribuído a cada tipo de domicílio $d \in D$.
* $C_d \in \mathbb{R}_+$: custo de construção de cada tipo de domicílio $d \in D$.
* $l \in \mathbb{R}_+$: percentual de espaço destinado a locais públicos.
* $f \in \mathbb{R}_+$: valor máximo de financiamento obtido do banco.




In [2]:
custo_de_demolicao_casa = 2000

numero_de_casas = 300

tamanho_lote_casa = 0.25

domicilios = ["simples", "duplos", "triplos", "quadruplos"]

tamanho_lote_domicilio = {
    "simples": 0.18,
    "duplos": 0.28,
    "triplos": 0.4,
    "quadruplos": 0.5
}

minimo_triplos_mais_quadruplos_perc = 0.25

minimo_simples_perc = 0.20

minimo_duplos_perc = 0.10

imposto_por_domicilio = {
    "simples": 1000,
    "duplos": 1900,
    "triplos": 2700,
    "quadruplos": 3400
}

custo_construcao_domicilios = {
    "simples": 50000,
    "duplos": 70000,
    "triplos": 130000,
    "quadruplos": 160000
}

locais_publicos_perc = 0.15

maximo_financiamento_banco = 15000000

## Declaração do objeto que representa o modelo matemático

In [3]:
model = pl.LpProblem("maximiza_arrecadacao", pl.LpMaximize)

## Variáveis de decisão

* $c\in \mathbb{Z}_+$: número de casas demolidas.
* $a_d \in \mathbb{Z}_+$: número de domicílios para cada tipo de domicílio $d \in D$.

In [4]:
casas_demolidas_var = pl.LpVariable("casas_demolidas", lowBound=0, cat='Integer')
domicilios_var = pl.LpVariable.dicts("domicilios", [domicilio for domicilio in domicilios], lowBound = 0, cat='Integer')

## Função objetivo

* Maximização do imposto arrecadado:\begin{equation}\max \sum \limits_{d \in D}(a_d \times I_d)\end{equation}

In [5]:
model += pl.lpSum([imposto_por_domicilio[domicilio] * domicilios_var[domicilio] for domicilio in domicilios])

## Restrições

* O gasto em demolição de casas e construção de domicílios não pode ultrapassar o financiamento máximo: \begin{equation}cd \times c + \sum \limits_{d \in D}(C_d \times a_d) ≤ f \end{equation}

In [6]:
model += casas_demolidas_var * custo_de_demolicao_casa +\
        pl.lpSum([custo_construcao_domicilios[domicilio] * domicilios_var[domicilio] for domicilio in domicilios])\
        <= maximo_financiamento_banco

* O número de casas demolidas não pode ultrapassar o número de casas: \begin{equation}c ≤ n \end{equation}

In [7]:
model += casas_demolidas_var <= 300

* A área liberada pela demolição de casas é maior ou igual a soma da área de domicílios construídos mais a área pública: \begin{equation} c \times t \ge c \times t \times l + \sum \limits_{d \in D}(a_d \times T_d) \end{equation}

In [8]:
model += casas_demolidas_var * tamanho_lote_casa \
            >= locais_publicos_perc * casas_demolidas_var * tamanho_lote_casa + \
                pl.lpSum([domicilios_var[domicilio] * tamanho_lote_domicilio[domicilio] for domicilio in domicilios])

* Percentual de domicílios simples, duplos, triplos e quádruplos devem satisfazer um percentual mínimo: \begin{equation} a_s \ge ps \times \sum\limits_{d \in D}(a_d) \end{equation}
\begin{equation} a_d \ge pd \times \sum\limits_{d \in D}(a_d) \end{equation}
\begin{equation} a_t + a_q \ge ptq \times \sum\limits_{d \in D}(a_d) \end{equation}

In [9]:
model += domicilios_var["simples"] >= minimo_simples_perc * pl.lpSum([domicilios_var[domicilio] for domicilio in domicilios])

model += domicilios_var["duplos"] >= minimo_duplos_perc * pl.lpSum([domicilios_var[domicilio] for domicilio in domicilios])

model += domicilios_var["triplos"] + domicilios_var["quadruplos"] >= \
            minimo_triplos_mais_quadruplos_perc * pl.lpSum([domicilios_var[domicilio] for domicilio in domicilios])

## Apresentação do modelo

In [10]:
model

maximiza_arrecadacao:
MAXIMIZE
1900*domicilios_duplos + 3400*domicilios_quadruplos + 1000*domicilios_simples + 2700*domicilios_triplos + 0
SUBJECT TO
_C1: 2000 casas_demolidas + 70000 domicilios_duplos
 + 160000 domicilios_quadruplos + 50000 domicilios_simples
 + 130000 domicilios_triplos <= 15000000

_C2: casas_demolidas <= 300

_C3: 0.2125 casas_demolidas - 0.28 domicilios_duplos
 - 0.5 domicilios_quadruplos - 0.18 domicilios_simples
 - 0.4 domicilios_triplos >= 0

_C4: - 0.2 domicilios_duplos - 0.2 domicilios_quadruplos
 + 0.8 domicilios_simples - 0.2 domicilios_triplos >= 0

_C5: 0.9 domicilios_duplos - 0.1 domicilios_quadruplos
 - 0.1 domicilios_simples - 0.1 domicilios_triplos >= 0

_C6: - 0.25 domicilios_duplos + 0.75 domicilios_quadruplos
 - 0.25 domicilios_simples + 0.75 domicilios_triplos >= 0

VARIABLES
0 <= casas_demolidas Integer
0 <= domicilios_duplos Integer
0 <= domicilios_quadruplos Integer
0 <= domicilios_simples Integer
0 <= domicilios_triplos Integer

## Resolução do modelo

In [11]:
status = model.solve()

## Impressão das soluções do problema

In [12]:
print('status: ', pl.LpStatus[status])
print('funcao objetivo: ', model.objective.value())
print('solucoes')
soma = sum([custo_construcao_domicilios[domicilio]*domicilios_var[domicilio].value() for domicilio in domicilios])
for domicilio in domicilios:
    print("Domicilios ", domicilio, ": ", domicilios_var[domicilio].value(), sep="")
print("Casas demolidas:", casas_demolidas_var.value())

status:  Optimal
funcao objetivo:  343700.0
solucoes
Domicilios simples: 36.0
Domicilios duplos: 98.0
Domicilios triplos: 45.0
Domicilios quadruplos: 0.0
Casas demolidas: 245.0
